In [116]:
import pandas as pd
import csv
import numpy as np

In [129]:
def read_text_file(path):
    data = []
    with open(path, 'r') as fd:
        reader = csv.reader(fd)
        for row in reader:
            current = row[0].split("\t")
            try:
                data.append((current[0], current[1]))
                print("Found", current[1])
            except:
                print("Sentiment Not Found")
            
            
    return data

In [130]:
DATASET_AMAZON = "data/amazon_cells_labelled.txt"
DATASET_YELP = "data/yelp_labelled.txt"
DATASET_IMDB = "data/imdb_labelled.txt"

DATASET = read_text_file(DATASET_AMAZON) + read_text_file(DATASET_YELP) + read_text_file(DATASET_IMDB)

Found 0
Sentiment Not Found
Found 1
Found 0
Found 1
Found 0
Sentiment Not Found
Found 1
Sentiment Not Found
Found 0
Found 1
Found 1
Found 0
Found 1
Sentiment Not Found
Found 1
Found 0
Found 1
Found 1
Found 0
Sentiment Not Found
Found 1
Found 0
Found 1
Found 0
Found 1
Found 1
Found 0
Found 0
Found 0
Sentiment Not Found
Found 1
Found 0
Found 0
Found 1
Found 0
Found 1
Found 0
Found 1
Found 0
Sentiment Not Found
Found 0
Found 1
Found 0
Found 1
Found 1
Found 0
Found 0
Found 1
Found 0
Found 0
Found 1
Found 1
Found 1
Found 1
Found 1
Found 0
Found 1
Found 1
Found 0
Found 0
Sentiment Not Found
Found 1
Found 0
Found 0
Found 0
Sentiment Not Found
Found 1
Found 1
Found 1
Found 0
Sentiment Not Found
Found 1
Found 1
Found 1
Found 1
Found 0
Found 1
Sentiment Not Found
Found 0
Found 1
Found 0
Sentiment Not Found
Found 0
Sentiment Not Found
Found 1
Found 1
Found 0
Found 1
Found 1
Sentiment Not Found
Found 1
Found 1
Found 0
Found 0
Found 1
Found 0
Found 0
Found 1
Found 0
Found 1
Sentiment Not Found
Foun

Found 1
Sentiment Not Found
Sentiment Not Found
Found 0
Found 0
Found 1
Found 1
Found 0
Sentiment Not Found
Found 1
Found 0
Found 1
Found 1
Found 0
Found 1
Found 0
Found 1
Found 1
Found 1
Sentiment Not Found
Found 0
Found 1
Sentiment Not Found
Found 0
Found 1
Sentiment Not Found
Found 0
Found 0
Sentiment Not Found
Found 1
Found 0
Found 1
Found 0
Found 0
Found 0
Found 0
Sentiment Not Found
Found 1
Found 1
Found 1
Found 0
Found 1
Found 1
Found 0
Found 1
Found 1
Found 0
Sentiment Not Found
Sentiment Not Found
Found 1
Sentiment Not Found
Found 0
Found 1
Found 0
Sentiment Not Found
Sentiment Not Found
Found 1
Found 1
Found 1
Found 1
Found 0
Sentiment Not Found
Found 0
Found 0
Found 1
Sentiment Not Found
Found 1
Found 0
Sentiment Not Found
Found 1
Found 1
Found 1
Sentiment Not Found
Found 1
Found 1
Found 0
Found 1
Sentiment Not Found
Found 1
Found 1
Sentiment Not Found
Sentiment Not Found
Found 1
Found 1
Found 1
Found 0
Found 0
Found 0
Found 0
Found 0
Sentiment Not Found
Sentiment Not Found


Found 1
Found 1
Sentiment Not Found
Found 1
Found 1
Found 1
Found 1
Sentiment Not Found
Sentiment Not Found
Found 0
Sentiment Not Found
Found 0
Found 0
Sentiment Not Found
Sentiment Not Found
Sentiment Not Found
Sentiment Not Found
Found 1
Sentiment Not Found
Found 0
Found 0
Found 0
Sentiment Not Found
Sentiment Not Found
Sentiment Not Found
Found 1
Found 1
Sentiment Not Found
Sentiment Not Found
Sentiment Not Found
Sentiment Not Found
Sentiment Not Found
Found 1
Sentiment Not Found
Found 1
Sentiment Not Found
Found 0
Found 0
Sentiment Not Found
Found 1
Found 1
Found 1
Sentiment Not Found
Sentiment Not Found
Found 1
Found 1
Found 1
Found 1
Found 0
Found 1
Found 1
Found 1
Found 1
Found 1
Found 1
Found 1
Sentiment Not Found
Found 0
Found 0
Found 0
Sentiment Not Found
Found 0
Sentiment Not Found
Sentiment Not Found
Found 0
Found 0
Sentiment Not Found
Sentiment Not Found
Sentiment Not Found
Sentiment Not Found
Sentiment Not Found
Found 0
Found 0
Found 0
Found 0
Found 0
Sentiment Not Found


In [131]:
DATASET = np.array(DATASET).transpose()

In [106]:
DATASET[0]

('So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0')

In [101]:
df = pd.DataFrame(DATASET, columns=['Text', 'Sentiment']) 

ValueError: Shape of passed values is (2737, 1), indices imply (2737, 2)

In [84]:
df.Text[0]

['So there is no way for me to plug it in here in the US unless I go by a converter.',
 '0']